In [1]:
# ============================
# IMPORT LIBRARY
# ============================
import simpy
import random
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
from dataclasses import dataclass
from typing import List, Tuple
import statistics

print("✅ Library berhasil diimport")

✅ Library berhasil diimport


In [2]:
# ============================
# KONFIGURASI SIMULASI
# ============================
@dataclass
class Config:
    """Konfigurasi parameter simulasi sistem piket IT Del"""
    
    # Parameter dasar
    NUM_MEJA: int = 60
    MAHASISWA_PER_MEJA: int = 3
    
    @property
    def TOTAL_OMPRENG(self):
        return self.NUM_MEJA * self.MAHASISWA_PER_MEJA
    
    # Alokasi petugas (total 7 orang)
    STAFF_LAUK: int = 3      # Memasukkan lauk ke ompreng
    STAFF_ANGKAT: int = 1     # Mengangkat ompreng (proses batch)
    STAFF_NASI: int = 3       # Menambahkan nasi
    
    # Waktu layanan (dalam menit)
    # Lauk: 30-60 detik = 0.5-1.0 menit
    LAUK_MIN: float = 0.5
    LAUK_MAX: float = 1.0
    
    # Angkat: 20-60 detik = 0.33-1.0 menit, membawa 4-7 ompreng
    ANGKAT_MIN: float = 0.33
    ANGKAT_MAX: float = 1.0
    ANGKAT_BATCH_MIN: int = 4
    ANGKAT_BATCH_MAX: int = 7
    
    # Nasi: 30-60 detik = 0.5-1.0 menit
    NASI_MIN: float = 0.5
    NASI_MAX: float = 1.0
    
    # Waktu mulai
    START_HOUR: int = 7
    START_MINUTE: int = 0
    
    # Seed untuk reproduktibilitas
    RANDOM_SEED: int = 42

print("✅ Konfigurasi simulasi siap")

✅ Konfigurasi simulasi siap


In [3]:
# ============================
# MODEL SIMULASI LENGKAP
# ============================
class SistemPiketITDel:
    """Model Discrete Event Simulation untuk sistem piket IT Del"""
    
    def __init__(self, config: Config):
        self.config = config
        self.env = simpy.Environment()
        
        # Sumber daya (resources)
        self.petugas_lauk = simpy.Resource(self.env, capacity=config.STAFF_LAUK)
        self.petugas_angkat = simpy.Resource(self.env, capacity=config.STAFF_ANGKAT)
        self.petugas_nasi = simpy.Resource(self.env, capacity=config.STAFF_NASI)
        
        # Antrian dan buffer
        self.antrian_lauk = simpy.Store(self.env)      # Antrian ompreng menunggu lauk
        self.antrian_nasi = simpy.Store(self.env)      # Antrian ompreng menunggu nasi
        self.buffer_angkat = []                          # Buffer ompreng siap angkat
        
        # Statistik
        self.statistics = {
            'ompreng_data': [],                          # Data setiap ompreng
            'waktu_tunggu': {'lauk': [], 'angkat': [], 'nasi': []},
            'waktu_layanan': {'lauk': [], 'angkat': [], 'nasi': []},
            'batch_data': {'ukuran': [], 'durasi': []},  # Data batch pengangkatan
            'antrian_lengths': {'lauk': [], 'angkat': [], 'nasi': []},  # Panjang antrian setiap event
            'utilization': {'lauk': [], 'angkat': [], 'nasi': []}      # Utilisasi petugas
        }
        
        # Waktu mulai simulasi
        self.start_time = datetime(2024, 1, 1, config.START_HOUR, config.START_MINUTE)
        
        # Set random seed
        random.seed(config.RANDOM_SEED)
        np.random.seed(config.RANDOM_SEED)
        
        # Counter untuk ompreng yang selesai
        self.ompreng_selesai = 0
        self.total_ompreng = config.TOTAL_OMPRENG
    
    def waktu_ke_jam(self, waktu_simulasi: float) -> datetime:
        """Konversi waktu simulasi (menit) ke datetime"""
        return self.start_time + timedelta(minutes=waktu_simulasi)
    
    def generate_lauk_time(self) -> float:
        """Generate waktu layanan lauk (0.5-1.0 menit)"""
        return random.uniform(self.config.LAUK_MIN, self.config.LAUK_MAX)
    
    def generate_angkat_time(self) -> float:
        """Generate waktu pengangkatan (0.33-1.0 menit)"""
        return random.uniform(self.config.ANGKAT_MIN, self.config.ANGKAT_MAX)
    
    def generate_batch_size(self) -> int:
        """Generate ukuran batch (4-7 ompreng)"""
        return random.randint(self.config.ANGKAT_BATCH_MIN, self.config.ANGKAT_BATCH_MAX)
    
    def generate_nasi_time(self) -> float:
        """Generate waktu penambahan nasi (0.5-1.0 menit)"""
        return random.uniform(self.config.NASI_MIN, self.config.NASI_MAX)
    
    def catat_antrian(self):
        """Mencatat panjang antrian saat ini"""
        self.statistics['antrian_lengths']['lauk'].append({
            'time': self.env.now,
            'length': len(self.antrian_lauk.items)
        })
        self.statistics['antrian_lengths']['angkat'].append({
            'time': self.env.now,
            'length': len(self.buffer_angkat)
        })
        self.statistics['antrian_lengths']['nasi'].append({
            'time': self.env.now,
            'length': len(self.antrian_nasi.items)
        })
    
    def proses_lauk(self, ompreng_id: int):
        """Proses untuk satu ompreng pada tahap lauk"""
        waktu_datang = self.env.now
        
        # 1. Masuk ke antrian lauk
        yield self.antrian_lauk.put(ompreng_id)
        self.catat_antrian()
        
        # 2. Minta petugas lauk
        with self.petugas_lauk.request() as request:
            yield request
            # Keluar dari antrian
            yield self.antrian_lauk.get()
            
            # Catat utilisasi
            self.statistics['utilization']['lauk'].append({
                'time': self.env.now,
                'in_use': self.petugas_lauk.count
            })
            
            # 3. Proses lauk
            service_time = self.generate_lauk_time()
            yield self.env.timeout(service_time)
            
            # Catat waktu layanan dan tunggu
            self.statistics['waktu_layanan']['lauk'].append(service_time)
            self.statistics['waktu_tunggu']['lauk'].append(self.env.now - waktu_datang - service_time)
        
        # 4. Pindah ke buffer angkat
        self.buffer_angkat.append({
            'id': ompreng_id, 
            'waktu_masuk_buffer': self.env.now
        })
        self.catat_antrian()
    
    def proses_angkat(self):
        """Proses pengangkatan ompreng secara batch"""
        while self.ompreng_selesai < self.total_ompreng:
            # Tentukan target ukuran batch
            target_batch = self.generate_batch_size()
            
            # Tunggu sampai terkumpul cukup ompreng di buffer
            while (len(self.buffer_angkat) < min(target_batch, 4) and 
                   len(self.buffer_angkat) + self.ompreng_selesai < self.total_ompreng):
                yield self.env.timeout(0.01)
                self.catat_antrian()
            
            # Proses batch jika ada ompreng
            if self.buffer_angkat:
                # Tentukan ukuran batch aktual
                batch_size = min(target_batch, len(self.buffer_angkat))
                
                # Hindari batch terlalu kecil jika masih banyak ompreng tersisa
                if batch_size < 4 and (self.ompreng_selesai + len(self.buffer_angkat) < self.total_ompreng):
                    yield self.env.timeout(0.01)
                    continue
                
                # Ambil batch dari buffer
                batch = self.buffer_angkat[:batch_size]
                self.buffer_angkat = self.buffer_angkat[batch_size:]
                
                # Catat waktu tunggu untuk setiap ompreng dalam batch
                waktu_mulai_angkat = self.env.now
                for item in batch:
                    self.statistics['waktu_tunggu']['angkat'].append(
                        waktu_mulai_angkat - item['waktu_masuk_buffer']
                    )
                
                # Proses pengangkatan
                with self.petugas_angkat.request() as request:
                    yield request
                    
                    self.statistics['utilization']['angkat'].append({
                        'time': self.env.now,
                        'in_use': self.petugas_angkat.count
                    })
                    
                    # Layanan angkat
                    service_time = self.generate_angkat_time()
                    yield self.env.timeout(service_time)
                    
                    # Catat statistik batch
                    self.statistics['waktu_layanan']['angkat'].append(service_time)
                    self.statistics['batch_data']['ukuran'].append(batch_size)
                    self.statistics['batch_data']['durasi'].append(self.env.now - waktu_mulai_angkat)
                
                # Kirim ompreng ke antrian nasi
                for item in batch:
                    yield self.antrian_nasi.put(item['id'])
                    # Proses nasi untuk setiap ompreng
                    self.env.process(self.proses_nasi(item['id']))
                
                self.catat_antrian()
            else:
                yield self.env.timeout(0.01)
                self.catat_antrian()
    
    def proses_nasi(self, ompreng_id: int):
        """Proses untuk satu ompreng pada tahap nasi"""
        waktu_masuk_antrian = self.env.now
        
        # Minta petugas nasi
        with self.petugas_nasi.request() as request:
            yield request
            
            self.statistics['utilization']['nasi'].append({
                'time': self.env.now,
                'in_use': self.petugas_nasi.count
            })
            
            # Proses nasi
            service_time = self.generate_nasi_time()
            yield self.env.timeout(service_time)
            
            # Catat statistik
            self.statistics['waktu_layanan']['nasi'].append(service_time)
            self.statistics['waktu_tunggu']['nasi'].append(
                self.env.now - waktu_masuk_antrian - service_time
            )
            
            # Ompreng selesai
            waktu_selesai = self.env.now
            self.statistics['ompreng_data'].append({
                'id': ompreng_id,
                'waktu_selesai': waktu_selesai,
                'jam_selesai': self.waktu_ke_jam(waktu_selesai)
            })
            
            self.ompreng_selesai += 1
            self.catat_antrian()
    
    def run_simulation(self):
        """Jalankan simulasi"""
        # Reset counter
        self.ompreng_selesai = 0
        self.buffer_angkat = []
        
        # Start proses angkat (berjalan terus)
        self.env.process(self.proses_angkat())
        
        # Generate semua ompreng
        for i in range(self.total_ompreng):
            self.env.process(self.proses_lauk(i))
        
        # Run simulasi
        self.env.run()
        
        return self.analyze_results()
    
    def analyze_results(self):
        """Analisis hasil simulasi"""
        if not self.statistics['ompreng_data']:
            return None, None
        
        df = pd.DataFrame(self.statistics['ompreng_data'])
        
        # Hitung statistik dasar
        results = {
            'total_ompreng': len(df),
            'waktu_selesai_terakhir': df['waktu_selesai'].max(),
            'jam_selesai_terakhir': self.waktu_ke_jam(df['waktu_selesai'].max()),
            'durasi_total': df['waktu_selesai'].max(),
            
            # Statistik waktu tunggu (konversi ke detik)
            'avg_tunggu_lauk': np.mean(self.statistics['waktu_tunggu']['lauk']) * 60 if self.statistics['waktu_tunggu']['lauk'] else 0,
            'avg_tunggu_angkat': np.mean(self.statistics['waktu_tunggu']['angkat']) * 60 if self.statistics['waktu_tunggu']['angkat'] else 0,
            'avg_tunggu_nasi': np.mean(self.statistics['waktu_tunggu']['nasi']) * 60 if self.statistics['waktu_tunggu']['nasi'] else 0,
            
            # Statistik waktu layanan (konversi ke detik)
            'avg_layanan_lauk': np.mean(self.statistics['waktu_layanan']['lauk']) * 60,
            'avg_layanan_angkat': np.mean(self.statistics['waktu_layanan']['angkat']) * 60,
            'avg_layanan_nasi': np.mean(self.statistics['waktu_layanan']['nasi']) * 60,
            'total_layanan_lauk': sum(self.statistics['waktu_layanan']['lauk']) * 60,
            'total_layanan_angkat': sum(self.statistics['waktu_layanan']['angkat']) * 60,
            'total_layanan_nasi': sum(self.statistics['waktu_layanan']['nasi']) * 60,
            
            # Statistik batch
            'avg_batch_size': np.mean(self.statistics['batch_data']['ukuran']) if self.statistics['batch_data']['ukuran'] else 0,
            'min_batch_size': np.min(self.statistics['batch_data']['ukuran']) if self.statistics['batch_data']['ukuran'] else 0,
            'max_batch_size': np.max(self.statistics['batch_data']['ukuran']) if self.statistics['batch_data']['ukuran'] else 0,
            'total_batch': len(self.statistics['batch_data']['ukuran']),
            'avg_batch_duration': np.mean(self.statistics['batch_data']['durasi']) * 60 if self.statistics['batch_data']['durasi'] else 0,
            
            # Throughput
            'throughput_per_jam': (self.total_ompreng / df['waktu_selesai'].max()) * 60,
            
            # Utilisasi (dalam persen)
            'utilisasi_lauk': 0,
            'utilisasi_angkat': 0,
            'utilisasi_nasi': 0,
            
            # Distribusi per jam
            'distribusi_jam': self.calculate_hourly_distribution(df)
        }
        
        # Hitung utilisasi
        total_waktu = results['waktu_selesai_terakhir']
        if total_waktu > 0:
            results['utilisasi_lauk'] = (sum(self.statistics['waktu_layanan']['lauk']) / (total_waktu * self.config.STAFF_LAUK)) * 100
            results['utilisasi_angkat'] = (sum(self.statistics['waktu_layanan']['angkat']) / (total_waktu * self.config.STAFF_ANGKAT)) * 100
            results['utilisasi_nasi'] = (sum(self.statistics['waktu_layanan']['nasi']) / (total_waktu * self.config.STAFF_NASI)) * 100
        
        return results, df
    
    def calculate_hourly_distribution(self, df):
        """Hitung distribusi penyelesaian per jam"""
        df['jam'] = df['jam_selesai'].dt.hour
        hourly = df.groupby('jam').size().reset_index(name='jumlah')
        return dict(zip(hourly['jam'], hourly['jumlah']))

print("✅ Model simulasi lengkap siap")

✅ Model simulasi lengkap siap


In [4]:
# ============================
# FUNGSI VISUALISASI (DIPERBAIKI)
# ============================
def run_scenario(config: Config):
    """Jalankan simulasi dengan konfigurasi tertentu"""
    print(f"\n{'='*60}")
    print(f"SIMULASI SISTEM PIKET IT DEL")
    print(f"{'='*60}")
    print(f"Jumlah Meja: {config.NUM_MEJA}")
    print(f"Mahasiswa per Meja: {config.MAHASISWA_PER_MEJA}")
    print(f"Total Ompreng: {config.TOTAL_OMPRENG}")
    print(f"\nAlokasi Petugas:")
    print(f"  - Lauk: {config.STAFF_LAUK} orang")
    print(f"  - Angkat: {config.STAFF_ANGKAT} orang")
    print(f"  - Nasi: {config.STAFF_NASI} orang")
    print(f"{'='*60}")

    # Jalankan model
    model = SistemPiketITDel(config)
    results, df = model.run_simulation()

    # Tampilkan hasil
    if results:
        print(f"\n📊 HASIL SIMULASI:")
        print(f"Jam mulai: {model.start_time.strftime('%H:%M')}")
        print(f"Jam selesai terakhir: {results['jam_selesai_terakhir'].strftime('%H:%M:%S')}")
        print(f"Durasi total: {results['durasi_total']:.2f} menit")
        print(f"Total batch pengangkatan: {results['total_batch']}")
        print(f"Throughput: {results['throughput_per_jam']:.1f} ompreng/jam")

        print(f"\n⏱️  STATISTIK WAKTU TUNGGU (detik):")
        print(f"  Lauk   : Rata-rata: {results['avg_tunggu_lauk']:.2f}")
        print(f"  Angkat : Rata-rata: {results['avg_tunggu_angkat']:.2f}")
        print(f"  Nasi   : Rata-rata: {results['avg_tunggu_nasi']:.2f}")

        print(f"\n⏱️  STATISTIK WAKTU LAYANAN (detik):")
        print(f"  Lauk   : Rata-rata: {results['avg_layanan_lauk']:.2f} | Total: {results['total_layanan_lauk']:.2f}")
        print(f"  Angkat : Rata-rata: {results['avg_layanan_angkat']:.2f} | Total: {results['total_layanan_angkat']:.2f}")
        print(f"  Nasi   : Rata-rata: {results['avg_layanan_nasi']:.2f} | Total: {results['total_layanan_nasi']:.2f}")

        print(f"\n📦 STATISTIK BATCH:")
        print(f"  Rata-rata ukuran: {results['avg_batch_size']:.1f} ompreng")
        print(f"  Ukuran min/max: {results['min_batch_size']:.0f} - {results['max_batch_size']:.0f} ompreng")
        print(f"  Rata-rata durasi proses batch: {results['avg_batch_duration']:.2f} detik")

        print(f"\n📈 UTILISASI PETUGAS:")
        print(f"  Lauk   : {results['utilisasi_lauk']:.1f}%")
        print(f"  Angkat : {results['utilisasi_angkat']:.1f}%")
        print(f"  Nasi   : {results['utilisasi_nasi']:.1f}%")

        print(f"\n🕐 DISTRIBUSI PER JAM:")
        for jam in sorted(results['distribusi_jam'].keys()):
            print(f"  Jam {jam:02d}:00 - {results['distribusi_jam'][jam]:3d} ompreng")

    return results, df, model


def visualize_results(df, model, results, scenario_name=""):
    """Visualisasi hasil simulasi dengan matplotlib"""
    if df is None or df.empty:
        print("Tidak ada data untuk divisualisasikan")
        return
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle(f'Hasil Simulasi Sistem Piket IT Del {scenario_name}', fontsize=14, fontweight='bold')

    # 1. Histogram Waktu Tunggu per Tahap
    ax = axes[0, 0]
    data_tunggu = []
    labels = []
    colors = ['blue', 'orange', 'green']
    
    if model.statistics['waktu_tunggu']['lauk']:
        data_tunggu.append(np.array(model.statistics['waktu_tunggu']['lauk']) * 60)
        labels.append('Lauk')
    if model.statistics['waktu_tunggu']['angkat']:
        data_tunggu.append(np.array(model.statistics['waktu_tunggu']['angkat']) * 60)
        labels.append('Angkat')
    if model.statistics['waktu_tunggu']['nasi']:
        data_tunggu.append(np.array(model.statistics['waktu_tunggu']['nasi']) * 60)
        labels.append('Nasi')
    
    if data_tunggu:
        ax.hist(data_tunggu, bins=15, label=labels, color=colors, alpha=0.7, edgecolor='black')
        ax.set_xlabel('Waktu Tunggu (detik)')
        ax.set_ylabel('Frekuensi')
        ax.set_title('Distribusi Waktu Tunggu per Tahap')
        ax.legend()
        ax.grid(True, alpha=0.3)

    # 2. Timeline Penyelesaian Kumulatif
    ax = axes[0, 1]
    df_sorted = df.sort_values('waktu_selesai')
    df_sorted['kumulatif'] = range(1, len(df_sorted) + 1)
    ax.plot(df_sorted['waktu_selesai'], df_sorted['kumulatif'], 'b-', linewidth=2)
    ax.set_xlabel('Waktu Simulasi (menit)')
    ax.set_ylabel('Ompreng Selesai')
    ax.set_title('Progress Kumulatif Penyelesaian')
    ax.grid(True, alpha=0.3)

    # 3. Distribusi Ukuran Batch
    ax = axes[0, 2]
    if model.statistics['batch_data']['ukuran']:
        ax.hist(model.statistics['batch_data']['ukuran'], bins=8, color='orange', edgecolor='black', alpha=0.7)
        ax.set_xlabel('Ukuran Batch (ompreng)')
        ax.set_ylabel('Frekuensi')
        ax.set_title('Distribusi Ukuran Batch')
        ax.grid(True, alpha=0.3)

    # 4. Utilisasi Petugas
    ax = axes[1, 0]
    if results:
        utilisasi = [
            results['utilisasi_lauk'],
            results['utilisasi_angkat'],
            results['utilisasi_nasi']
        ]
        labels_util = ['Lauk', 'Angkat', 'Nasi']
        colors_util = ['blue', 'orange', 'green']
        bars = ax.bar(labels_util, utilisasi, color=colors_util, alpha=0.7, edgecolor='black')
        ax.set_ylabel('Utilisasi (%)')
        ax.set_title('Utilisasi Petugas')
        ax.set_ylim([0, 100])
        ax.grid(True, alpha=0.3, axis='y')
        
        # Tambah nilai di atas bar
        for bar, val in zip(bars, utilisasi):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{val:.1f}%', 
                    ha='center', va='bottom', fontsize=10)

    # 5. Panjang Antrian Sepanjang Waktu
    ax = axes[1, 1]
    if model.statistics['antrian_lengths']['lauk']:
        waktu = [d['time'] for d in model.statistics['antrian_lengths']['lauk']]
        lauk = [d['length'] for d in model.statistics['antrian_lengths']['lauk']]
        
        # Cari data angkat dan nasi terdekat untuk setiap waktu
        angkat = []
        nasi = []
        for t in waktu:
            a = next((d['length'] for d in model.statistics['antrian_lengths']['angkat'] if abs(d['time'] - t) < 0.1), 0)
            angkat.append(a)
            n = next((d['length'] for d in model.statistics['antrian_lengths']['nasi'] if abs(d['time'] - t) < 0.1), 0)
            nasi.append(n)
        
        ax.plot(waktu, lauk, 'b-', label='Antrian Lauk', linewidth=1.5)
        ax.plot(waktu, angkat, 'orange', label='Buffer Angkat', linewidth=1.5)
        ax.plot(waktu, nasi, 'g-', label='Antrian Nasi', linewidth=1.5)
        ax.set_xlabel('Waktu (menit)')
        ax.set_ylabel('Panjang Antrian')
        ax.set_title('Dinamika Antrian Sepanjang Waktu')
        ax.legend()
        ax.grid(True, alpha=0.3)

    # 6. Distribusi Penyelesaian per Menit
    ax = axes[1, 2]
    df['menit'] = df['jam_selesai'].dt.minute + df['jam_selesai'].dt.hour * 60 - 420  # 420 = 7 jam * 60
    per_menit = df['menit'].value_counts().sort_index()
    ax.bar(per_menit.index, per_menit.values, width=0.8, color='green', alpha=0.7, edgecolor='black')
    ax.set_xlabel('Menit Ke-')
    ax.set_ylabel('Jumlah Ompreng')
    ax.set_title('Distribusi Penyelesaian per Menit')
    ax.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.show()


def sensitivity_analysis():
    """Analisis sensitivitas dengan variasi parameter"""
    scenarios = [
        # Skenario 1: Konfigurasi dasar (3-1-3)
        {"staff_lauk": 3, "staff_angkat": 1, "staff_nasi": 3, "nama": "Dasar (3-1-3)"},
        
        # Skenario 2: Lebih banyak petugas angkat
        {"staff_lauk": 2, "staff_angkat": 2, "staff_nasi": 3, "nama": "Angkat 2 (2-2-3)"},
        
        # Skenario 3: Lebih banyak petugas lauk
        {"staff_lauk": 4, "staff_angkat": 1, "staff_nasi": 2, "nama": "Lauk 4 (4-1-2)"},
        
        # Skenario 4: Seimbang semua
        {"staff_lauk": 3, "staff_angkat": 2, "staff_nasi": 2, "nama": "Seimbang (3-2-2)"},
        
        # Skenario 5: Petugas nasi lebih banyak
        {"staff_lauk": 2, "staff_angkat": 1, "staff_nasi": 4, "nama": "Nasi 4 (2-1-4)"},
        
        # Skenario 6: Angkat maksimal
        {"staff_lauk": 2, "staff_angkat": 3, "staff_nasi": 2, "nama": "Angkat 3 (2-3-2)"},
    ]
    
    results_summary = []

    for i, scenario in enumerate(scenarios):
        print(f"\n{'='*60}")
        print(f"SKENARIO {i+1}: {scenario['nama']}")
        print(f"{'='*60}")
        
        config = Config(
            STAFF_LAUK=scenario["staff_lauk"],
            STAFF_ANGKAT=scenario["staff_angkat"],
            STAFF_NASI=scenario["staff_nasi"]
        )

        model = SistemPiketITDel(config)
        results, df = model.run_simulation()

        if results:
            results_summary.append({
                'Skenario': i+1,
                'Konfigurasi': scenario['nama'],
                'Lauk/ Angkat/ Nasi': f"{scenario['staff_lauk']}-{scenario['staff_angkat']}-{scenario['staff_nasi']}",
                'Jam Selesai': results['jam_selesai_terakhir'].strftime('%H:%M:%S'),
                'Durasi (menit)': f"{results['durasi_total']:.1f}",
                'Batch': results['total_batch'],
                'Throughput (/jam)': f"{results['throughput_per_jam']:.0f}",
                'Tunggu Lauk (dtk)': f"{results['avg_tunggu_lauk']:.1f}",
                'Tunggu Angkat (dtk)': f"{results['avg_tunggu_angkat']:.1f}",
                'Tunggu Nasi (dtk)': f"{results['avg_tunggu_nasi']:.1f}",
                'Util Lauk (%)': f"{results['utilisasi_lauk']:.1f}",
                'Util Angkat (%)': f"{results['utilisasi_angkat']:.1f}",
                'Util Nasi (%)': f"{results['utilisasi_nasi']:.1f}"
            })

    # Tampilkan tabel perbandingan
    print(f"\n{'='*100}")
    print(f"{'PERBANDINGAN SEMUA SKENARIO':^100}")
    print(f"{'='*100}")

    summary_df = pd.DataFrame(results_summary)
    print(summary_df.to_string(index=False))
    
    # Visualisasi perbandingan
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Perbandingan Kinerja Antar Skenario', fontsize=14, fontweight='bold')
    
    # Durasi
    ax = axes[0, 0]
    x = range(1, len(results_summary) + 1)
    durasi = [float(s['Durasi (menit)']) for s in results_summary]
    labels = [s['Konfigurasi'] for s in results_summary]
    bars = ax.bar(x, durasi, color='skyblue', edgecolor='black')
    ax.set_xlabel('Skenario')
    ax.set_ylabel('Durasi (menit)')
    ax.set_title('Durasi Penyelesaian')
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Throughput
    ax = axes[0, 1]
    throughput = [float(s['Throughput (/jam)']) for s in results_summary]
    bars = ax.bar(x, throughput, color='lightgreen', edgecolor='black')
    ax.set_xlabel('Skenario')
    ax.set_ylabel('Throughput (ompreng/jam)')
    ax.set_title('Throughput')
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Waktu Tunggu Rata-rata
    ax = axes[1, 0]
    width = 0.25
    x_pos = np.arange(len(results_summary))
    
    tunggu_lauk = [float(s['Tunggu Lauk (dtk)']) for s in results_summary]
    tunggu_angkat = [float(s['Tunggu Angkat (dtk)']) for s in results_summary]
    tunggu_nasi = [float(s['Tunggu Nasi (dtk)']) for s in results_summary]
    
    ax.bar(x_pos - width, tunggu_lauk, width, label='Lauk', color='blue', alpha=0.7)
    ax.bar(x_pos, tunggu_angkat, width, label='Angkat', color='orange', alpha=0.7)
    ax.bar(x_pos + width, tunggu_nasi, width, label='Nasi', color='green', alpha=0.7)
    ax.set_xlabel('Skenario')
    ax.set_ylabel('Waktu Tunggu (detik)')
    ax.set_title('Waktu Tunggu per Tahap')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Utilisasi
    ax = axes[1, 1]
    util_lauk = [float(s['Util Lauk (%)']) for s in results_summary]
    util_angkat = [float(s['Util Angkat (%)']) for s in results_summary]
    util_nasi = [float(s['Util Nasi (%)']) for s in results_summary]
    
    ax.bar(x_pos - width, util_lauk, width, label='Lauk', color='blue', alpha=0.7)
    ax.bar(x_pos, util_angkat, width, label='Angkat', color='orange', alpha=0.7)
    ax.bar(x_pos + width, util_nasi, width, label='Nasi', color='green', alpha=0.7)
    ax.set_xlabel('Skenario')
    ax.set_ylabel('Utilisasi (%)')
    ax.set_title('Utilisasi Petugas')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim([0, 100])
    
    plt.tight_layout()
    plt.show()
    
    return summary_df

print("✅ Fungsi visualisasi siap (telah diperbaiki)")

✅ Fungsi visualisasi siap (telah diperbaiki)
